# Optimization Methods For Data Science
## Final Project - Part 2: SVM

Géraldine V. Maurer, Viktoriia Vlasenko

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import random
from scipy.optimize import minimize
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
random.seed(42)
np.random.seed(42)

warnings.filterwarnings('ignore')

from functions_1j_maurer_vlasenko import *

### Import Data

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/gmaurer08/Optimization-Final-Project/refs/heads/main/AGE_PREDICTION.csv")
data.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,gt
0,2.686191,-0.989465,-0.920503,1.607427,-0.896248,1.118974,-0.969456,1.811707,2.560955,3.803463,...,-0.862891,-0.909545,-0.915361,-0.952061,-0.989461,1.911855,1.409705,2.303997,-0.981840,54
1,-0.887917,4.915272,-0.939446,-0.343677,-0.964685,-0.478649,4.342395,-0.332870,-0.768041,-0.815375,...,-0.939201,-0.965917,-0.969461,-0.934799,5.304822,0.934790,-0.410701,0.284690,4.919212,18
2,-0.923215,2.746968,-0.918085,0.047804,-0.908587,-0.451752,2.984481,0.535007,-0.591029,-0.324043,...,-0.809726,-0.929934,-0.891814,-0.881796,3.415373,1.044108,-0.442615,0.033648,2.628199,26
3,-0.268866,-0.408416,-0.935145,0.731800,-0.922438,0.221781,-0.046606,1.149634,0.592136,1.357959,...,-0.834968,-0.937475,-0.917737,-0.929519,-0.226282,1.608048,0.276169,1.246468,-0.363367,33
4,0.529231,-0.829957,-0.897425,0.921280,-0.865304,0.331018,-0.644940,1.296097,1.166863,2.036034,...,-0.775411,-0.881967,-0.864018,-0.908001,-0.784495,1.329586,0.547925,1.195395,-0.810089,35


### Data Preparation

In [3]:
# Separate features and target
feature_cols = [col for col in data.columns if col.startswith('feat')]
X = data[feature_cols].values
y = data['gt'].values

print(f"Features: {len(feature_cols)} columns")
print(f"Data shape: {X.shape}")
print(f"Target range: {y.min():.2f}-{y.max():.2f}")

# Split data into train/test sets
n_total = len(X)
n_train = int(0.8*n_total)    # 80% for training (used with CV inside)
# Remaining 20% for testing

# Shuffle indices
indices = np.random.permutation(n_total)
train_idx = indices[:n_train]
test_idx = indices[n_train:]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

Features: 32 columns
Data shape: (20475, 32)
Target range: 10.00-89.00
Train set: (16380, 32)
Test set: (4095, 32)


### Find the best hyperparameters with Cross-Validation

In [ ]:
# Find best hyperparameters using cross-validation
best_parameters, search_results = hyperparameter_search(X_train, y_train)

Starting hyperparameter search

[1/24] Testing configuration:
Layers: [64, 32] (hidden)
Activation: tanh
Lambda: 0.001

Starting 3-fold cross-validation...

Fold 1/3
Training network with architecture: [32, 64, 32, 1]
Activation: tanh, Lambda: 0.001
Initial loss: 1662.687561
Final loss: 86.543487
Optimization successful: False
Number of iterations: 1000
Train Loss: 83.8020, Val Loss: 104.0971
Train MAPE: 21.4188%, Val MAPE: 24.0400%

Fold 2/3
Training network with architecture: [32, 64, 32, 1]
Activation: tanh, Lambda: 0.001
Initial loss: 1656.307293
Final loss: 85.941984
Optimization successful: False
Number of iterations: 1000
Train Loss: 83.5605, Val Loss: 104.5303
Train MAPE: 21.4481%, Val MAPE: 24.0472%

Fold 3/3
Training network with architecture: [32, 64, 32, 1]
Activation: tanh, Lambda: 0.001
Initial loss: 1655.990383
Final loss: 85.076143
Optimization successful: False
Number of iterations: 1000
Train Loss: 82.1874, Val Loss: 104.1293
Train MAPE: 21.4136%, Val MAPE: 24.2008%
R

### Training the final model with the best hyperparameters

In [ ]:
import time
np.random.seed(42)

# Train final model with best parameters
start_time = time.time()
final_weights, final_biases, optimization_result, initial_loss, final_loss = train_network(
    X_train, y_train,
    layer_sizes=best_parameters['layers'],
    activation=best_parameters['activation'],
    lambda_reg=best_parameters['lambda_reg'],
    method='L-BFGS-B',
    maxiter=1000,
    seed=42
)
# Compute time difference from start
end_time = time.time()
optimization_time = end_time - start_time

# Make predictions on all datasets
y_train_pred = predict(X_train, final_weights, final_biases, best_parameters['activation'])
y_test_pred = predict(X_test, final_weights, final_biases, best_parameters['activation'])

# Compute Mean Squared Error
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Compute Mean Absolute Percentage Error
train_mape = MAPE(y_train, y_train_pred)
test_mape = MAPE(y_test, y_test_pred)

# Output results
print(f"1. Number of layers L chosen: {len(best_parameters['layers'])-1}")
print(f"2. Number of neurons N chosen: {max(best_parameters['layers'][1:-1]) if len(best_parameters['layers'])>2 else best_parameters['layers'][1]}")
print(f"3. Value of lambda chosen: {best_parameters['lambda_reg']}")
print(f"4. Other hyperparameters: activation = {best_parameters['activation']}, maxiter = 1000")
print(f"5. Optimization solver: L-BFGS-B")
print(f"6. Number of iterations: {optimization_result.nit}")
print(f"7. Optimization time: {optimization_time:.2f} seconds")
print(f"8. Training Error (without regularization): {train_mse:.6f}")
print(f"9. Test Error: {test_mse:.6f}")

Training network with architecture: [32, 64, 32, 16, 1]
Activation: sigmoid, Lambda: 0.01
Initial loss: 1675.734443
Final loss: 98.689045
Optimization successful: False
Number of iterations: 1000
1. Number of layers L chosen: 4
2. Number of neurons N chosen: 64
3. Value of lambda chosen: 0.01
4. Other hyperparameters: activation = sigmoid, maxiter = 1000
5. Optimization solver: L-BFGS-B
6. Number of iterations: 1000
7. Optimization time: 242.45 seconds
8. Training Error (without regularization): 95.003767
9. Test Error: 91.734070


### Save Results

In [ ]:
# Save detailed results
results_summary = {
    'best_config': best_parameters,
    'optimization_result': {
        'success': optimization_result.success,
        'message': optimization_result.message,
        'iterations': optimization_result.nit,
        'optimization_time': optimization_time,
        'initial_loss': initial_loss,
        'final_loss': final_loss
    },
    'performance_metrics': {
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_mape': train_mape,
        'test_mape': test_mape
    },
    'hyperparameter_search_results': search_results
}

In [ ]:
import pickle
with open("results_summary.pkl", "wb") as f:
    pickle.dump(results_summary,f)
f.close()

In [ ]:
results_summary

{'best_config': {'layers': [32, 64, 32, 16, 1],
  'activation': 'sigmoid',
  'lambda_reg': 0.01},
 'optimization_result': {'success': False,
  'message': 'STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT',
  'iterations': 1000,
  'optimization_time': 271.80328464508057,
  'initial_loss': np.float64(1734.996120744687),
  'final_loss': np.float64(98.68810268057803)},
 'performance_metrics': {'train_mse': 95.00780876569202,
  'test_mse': 91.72251600512398,
  'train_mape': np.float64(23.31217961550376),
  'test_mape': np.float64(22.763067005720114)},
 'hyperparameter_search_results': [{'layers': [32, 64, 32, 1],
   'activation': 'tanh',
   'lambda_reg': 0.001,
   'avg_val_loss': np.float64(105.28236443534229),
   'avg_val_mape': np.float64(24.15918092646488),
   'avg_train_loss': np.float64(83.06689650781153),
   'avg_train_mape': np.float64(21.368623353417735)},
  {'layers': [32, 64, 32, 1],
   'activation': 'tanh',
   'lambda_reg': 0.01,
   'avg_val_loss': np.float64(96.57989694851581),
   'a